## ACL : fetch citation graph from semantic scholar



In [61]:
import pandas as pd
from pys2 import (
    _get_redshift_connection,
    _load_dataframe_to_redshift,
)
import glob, os
from operator import itemgetter
from tqdm import tqdm
import logging
from urllib.parse import urlparse
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
logging.basicConfig(level=logging.INFO)


In [3]:
df_acl = pd.read_parquet('acl-publication-info.74k.parquet')

In [4]:
df_acl

,acl_id,abstract,full_text,corpus_paper_id,pdf_hash,numcitedby,url,publisher,address,year,...,booktitle,author,title,pages,doi,number,volume,journal,editor,isbn
0,O02-2002,There is a need to measure word similarity whe...,There is a need to measure word similarity whe...,18022704,0b09178ac8d17a92f16140365363d8df88c757d0,14,https://aclanthology.org/O02-2002,None,None,2002,...,International Journal of Computational Linguis...,"Chen, Keh-Jiann and\nYou, Jia-Ming",A Study on Word Similarity using Context Vecto...,37--58,None,None,None,None,None,None
1,L02-1310,,,8220988,8d5e31610bc82c2abc86bc20ceba684c97e66024,93,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,Proceedings of the Third International Confere...,"Mihalcea, Rada F.",Bootstrapping Large Sense Tagged Corpora,None,None,None,None,None,None,None
2,R13-1042,Thread disentanglement is the task of separati...,Thread disentanglement is the task of separati...,16703040,3eb736b17a5acb583b9a9bd99837427753632cdb,10,https://aclanthology.org/R13-1042,"INCOMA Ltd. Shoumen, BULGARIA","Hissar, Bulgaria",2013,...,Proceedings of the International Conference Re...,"Jamison, Emily and\nGurevych, Iryna","Headerless, Quoteless, but not Hopeless? Using...",327--335,None,None,None,None,None,None
3,W05-0819,"In this paper, we describe a word alignment al...","In this paper, we describe a word alignment al...",1215281,b20450f67116e59d1348fc472cfc09f96e348f55,15,https://aclanthology.org/W05-0819,Association for Computational Linguistics,"Ann Arbor, Michigan",2005,...,Proceedings of the {ACL} Workshop on Building ...,"Aswani, Niraj and\nGaizauskas, Robert",Aligning Words in {E}nglish-{H}indi Parallel C...,115--118,None,None,None,None,None,None
4,L02-1309,,,18078432,011e943b64a78dadc3440674419821ee080f0de3,12,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,Proceedings of the Third International Confere...,"Suyaga, Fumiaki and\nTakezawa, Toshiyuki and...",Proposal of a very-large-corpus acquisition me...,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73280,P99-1002,This paper describes recent progress and the a...,This paper describes recent progress and the a...,715160,ab17a01f142124744c6ae425f8a23011366ec3ee,11,https://aclanthology.org/P99-1002,Association for Computational Linguistics,"College Park, Maryland, USA",1999,...,Proceedings of the 37th Annual Meeting of the ...,"Furui, Sadaoki",Automatic Speech Recognition and Its Applicati...,11--20,10.3115/1034678.1034680,None,None,None,None,None
73281,P00-1009,We present an LFG-DOP parser which uses fragme...,We present an LFG-DOP parser which uses fragme...,1356246,ad005b3fd0c867667118482227e31d9378229751,12,https://aclanthology.org/P00-1009,Association for Computational Linguistics,Hong Kong,2000,...,Proceedings of the 38th Annual Meeting of the ...,"Bod, Rens",An Improved Parser for Data-Oriented Lexical-F...,61--68,10.3115/1075218.1075227,None,None,None,None,None
73282,P99-1056,The processes through which readers evoke ment...,The processes through which readers evoke ment...,7277828,924cf7a4836ebfc20ee094c30e61b949be049fb6,14,https://aclanthology.org/P99-1056,Association for Computational Linguistics,"College Park, Maryland, USA",1999,...,Proceedings of the 37th Annual Meeting of the ...,"Lange, Marielle and\nContent, Alain",The grapho-phonological system of written {F}r...,436--442,10.3115/1034678.1034745,None,None,None,None,None
73283,P99-1051,This paper examines the extent to which verb d...,This paper examines the extent to which verb d...,1829043,6b1f6f28ee36de69e8afac39461ee1158cd4d49a,92,https://aclanthology.org/P99-1051,Association for Computational Linguistics,"College Park, Maryland, USA",1999,...,Proceedings of the 37th Annual Meeting of the ...,"Lapata, Maria",Acquiring Le

In [5]:
_load_dataframe_to_redshift(df_acl['corpus_paper_id'].to_frame(), 'public.temp_acl_ids', create_table=True, write_privileges=True)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
/net/nfs2.s2-research/shaurya/sw/miniconda3/envs/s2jupy/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
INFO:pys2:Truncating table public.temp_acl_ids
INFO:pys2:Loading new data from S3 into public.temp_acl_ids
INFO:pys2:Done loading public.temp_acl_ids


In [6]:
query = f"""SELECT cps.id, cps.citingpaperid, cps.citedpaperid
                        FROM content_ext.citation_es as cps 
                        join public.temp_acl_ids as tcs
                        on cps.citingpaperid=tcs.corpus_paper_id;"""
df_res = pd.read_sql(query, _get_redshift_connection(write_privileges=True))

/net/nfs2.s2-research/shaurya/sw/miniconda3/envs/s2jupy/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
df_citingpaperid_acl = df_res 

In [8]:
query = f"""SELECT cps.id, cps.citingpaperid, cps.citedpaperid
                        FROM content_ext.citation_es as cps 
                        join public.temp_acl_ids as tcs
                        on cps.citedpaperid=tcs.corpus_paper_id;"""
df_citedpaperid_acl = pd.read_sql(query, _get_redshift_connection(write_privileges=True))

/net/nfs2.s2-research/shaurya/sw/miniconda3/envs/s2jupy/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [9]:
df_citedpaperid_acl = df_citedpaperid_acl.drop_duplicates().dropna().astype('int64')

In [10]:
df_citingpaperid_acl = df_citingpaperid_acl.drop_duplicates().dropna().astype('int64')

In [11]:
# sanity check for a paper with no citations

df_citedpaperid_acl[df_citedpaperid_acl.citedpaperid==16973602]

,id,citingpaperid,citedpaperid


In [12]:
df_acl[df_acl.corpus_paper_id==16973602]

,acl_id,abstract,full_text,corpus_paper_id,pdf_hash,numcitedby,url,publisher,address,year,...,booktitle,author,title,pages,doi,number,volume,journal,editor,isbn
67533,Y03-1027,The Semantic Knowledge-base of Contemporary Ch...,The Semantic Knowledge-base of Contemporary Ch...,16973602,f1e2687c45976eaac25e9de49d05c7b00546e1e6,0,https://aclanthology.org/Y03-1027,COLIPS PUBLICATIONS,"Sentosa, Singapore",2003,...,Proceedings of the 17th Pacific Asia Conferenc...,"Wang, Hui and\nYu, Shiwen",A Large-scale Lexical Semantic Knowledge-base ...,243--250,http://hdl.handle.net/2065/12277,None,None,None,None,None


In [13]:
df_citedpaperid_acl

,id,citingpaperid,citedpaperid
0,868703600,52046747,16228715
1,3216252045,229723601,1373518
2,437616166,16841192,629094
3,3349792001,235196103,19121210
4,3313665988,233365331,218974137
...,...,...,...
6756920,2497763129,63858766,7691746
6757426,2764904898,64329213,680537
6757794,3302005661,61861893,7287507
6757899,2504541468,209051634,4357791


In [14]:
df_citedpaperid_acl['is_citedpaperid_acl']=True

In [15]:
df_citingpaperid_acl['is_citingpaperid_acl']=True

In [16]:
set_acl_ids = set(df_acl.corpus_paper_id)

def is_id_acl(set_acl_ids, corpus_paper_id):
    if corpus_paper_id in set_acl_ids:
        return True
    return False

print(is_id_acl(set_acl_ids, 15220800)) # false
print(is_id_acl(set_acl_ids, 9278872)) # true

False
True


In [17]:
df_citingpaperid_acl["is_citedpaperid_acl"] = df_citingpaperid_acl.apply(
        lambda row: is_id_acl(set_acl_ids, row["citedpaperid"]), axis=1
    )

In [18]:
df_citingpaperid_acl

,id,citingpaperid,citedpaperid,is_citingpaperid_acl,is_citedpaperid_acl
0,3185814163,12438177,203672231,True,False
1,2083146504,201692196,646594,True,True
4,3337893297,234347863,17519114,True,False
5,51691323,2101161,13609860,True,False
7,3151767080,226283687,196211238,True,True
...,...,...,...,...,...
5457912,3196087989,210063434,216005337,True,False
5458425,3428018908,106054,215835800,True,False
5458635,500144049,53235202,13858533,True,True
5458855,981264328,2376390,65191799,True,False


In [20]:
df_citedpaperid_acl["is_citingpaperid_acl"] = df_citedpaperid_acl.apply(
        lambda row: is_id_acl(set_acl_ids, row["citingpaperid"]), axis=1
    )

In [21]:
df_citedpaperid_acl

,id,citingpaperid,citedpaperid,is_citedpaperid_acl,is_citingpaperid_acl
0,868703600,52046747,16228715,True,False
1,3216252045,229723601,1373518,True,False
2,437616166,16841192,629094,True,False
3,3349792001,235196103,19121210,True,False
4,3313665988,233365331,218974137,True,True
...,...,...,...,...,...
6756920,2497763129,63858766,7691746,True,False
6757426,2764904898,64329213,680537,True,False
6757794,3302005661,61861893,7287507,True,False
6757899,2504541468,209051634,4357791,True,False


In [23]:
df_full_acl_graph = pd.concat([df_citedpaperid_acl, df_citingpaperid_acl])

In [32]:
df_only_acl = df_full_acl_graph.loc[(df_full_acl_graph['is_citedpaperid_acl'] == True) & (df_full_acl_graph['is_citingpaperid_acl'] == True)]

In [40]:
df_only_acl[['citingpaperid', 'citedpaperid']].drop_duplicates()

,citingpaperid,citedpaperid
4,233365331,218974137
5,3841628,18998986
8,218974058,204896994
12,53242563,15453873
14,1009868,680757
...,...,...
6717667,16922546,2824500
6742226,770143,2791941
6743918,53981714,52967399
6751741,11495042,6243853


# Plot graph with edges and nodes

In [35]:
import networkx as nx
G = nx.Graph()

In [36]:
G = nx.from_pandas_edgelist(df_only_acl, 'citingpaperid', 'citedpaperid')

In [39]:
print(nx.info(G))

Graph with 64185 nodes and 665862 edges


/tmp/ipykernel_280237/2606185536.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(G))


In [41]:
density = nx.density(G)
print("Network density:", density)


Network density: 0.00032326143897407365


In [ ]:
components = nx.connected_components(G)
largest_component = max(components, key=len)

# Create a "subgraph" of just the largest component
# Then calculate the diameter of the subgraph, just like you did with density.


subgraph = G.subgraph(largest_component)
diameter = nx.diameter(subgraph)
print("Network diameter of largest component:", diameter)

In [43]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')

In [59]:
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)

In [49]:
df_degree = pd.DataFrame(sorted_degree, columns =['corpus_paper_id', 'degree'])

In [50]:
df_degree

,corpus_paper_id,degree
0,52967399,7877
1,11080756,5785
2,1957433,4548
3,794019,2890
4,3626819,2434
...,...,...
64180,219310311,1
64181,18106511,1
64182,219306848,1
64183,52010923,1


In [115]:
df_top100= df_acl.merge(df_degree, on='corpus_paper_id')[['degree', 'title', 'year','publisher']].sort_values(by=['degree'], ascending=False)[:1000]

In [116]:
df_top100[:50]

,degree,title,year,publisher
37460,7877,{BERT}: Pre-training of Deep Bidirectional Transformers for Language Understanding,2019,Association for Computational Linguistics
83,5785,{B}leu: a Method for Automatic Evaluation of Machine Translation,2002,Association for Computational Linguistics
47979,4548,{G}lo{V}e: Global Vectors for Word Representation,2014,Association for Computational Linguistics
14061,2890,{M}oses: Open Source Toolkit for Statistical Machine Translation,2007,Association for Computational Linguistics
36834,2434,Deep Contextualized Word Representations,2018,Association for Computational Linguistics
36356,2348,Building a Large Annotated Corpus of {E}nglish: The {P}enn {T}reebank,1993,MIT Press
41111,2021,Neural Machine Translation of Rare Words with Subword Units,2016,Association for Computational Linguistics
43843,1947,A Systematic Comparison of Various Statistical Alignment Models,2003,None
47920,1702,Minimum Error Rate Training in Statistical Machine Translation,2003,Association for Computational Linguistics
55647,1674,The Mathematics of Statistical Machine Translation: Parameter Estimation,1993,MIT Press


In [81]:
import plotly.express as px
df = px.data.tips()
fig = px.histogram(df_top100, y="year")
fig.update_yaxes(categoryorder='category ascending')
fig.show()


In [84]:
import plotly.express as px
df = px.data.tips()
fig = px.histogram(df_acl, x="year")
fig.update_xaxes(categoryorder='category ascending')
fig.show()


In [111]:
df_acl.columns

Index(['acl_id', 'abstract', 'full_text', 'corpus_paper_id', 'pdf_hash',
       'numcitedby', 'url', 'publisher', 'address', 'year', 'month',
       'booktitle', 'author', 'title', 'pages', 'doi', 'number', 'volume',
       'journal', 'editor', 'isbn'],
      dtype='object')

In [95]:
df_acl_degree = df_acl.merge(df_degree, on='corpus_paper_id').sort_values(by=['degree'], ascending=False)

In [112]:
df_acl_degree[['degree', 'title', 'year', 'numcitedby','publisher','journal']].sort_values(by=['numcitedby'], ascending=False)[:50]

,degree,title,year,numcitedby,publisher,journal
37460,7877,{BERT}: Pre-training of Deep Bidirectional Transformers for Language Understanding,2019,37353,Association for Computational Linguistics,None
47979,4548,{G}lo{V}e: Global Vectors for Word Representation,2014,23467,Association for Computational Linguistics,None
83,5785,{B}leu: a Method for Automatic Evaluation of Machine Translation,2002,17139,Association for Computational Linguistics,None
3484,1608,Learning Phrase Representations using {RNN} Encoder{--}Decoder for Statistical Machine Translation,2014,15755,Association for Computational Linguistics,None
55743,1416,{W}ord{N}et: A Lexical Database for {E}nglish,1992,14182,None,None
47944,1492,Convolutional Neural Networks for Sentence Classification,2014,10466,Association for Computational Linguistics,None
6440,719,Thumbs up? Sentiment Classification using Machine Learning Techniques,2002,8680,Association for Computational Linguistics,None
36834,2434,Deep Contextualized Word Representations,2018,8345,Association for Computational Linguistics,None
36356,2348,Building a Large Annotated Corpus of {E}nglish: The {P}enn {T}reebank,1993,8262,MIT Press,Computational Linguistics
57671,1576,{ROUGE}: A Package for Automatic Evaluation of Summaries,2004,7250,Association for Computational Linguistics,None


In [104]:
df_acl_degree[['degree', 'numcitedby', 'year']].corr()

,degree,numcitedby
degree,1.000000,0.905408
numcitedby,0.905408,1.000000


In [118]:
df_only_acl.to_parquet('acl_onlygraph.parquet')

In [119]:
df_full_acl_graph.to_parquet('acl_full_citations.parquet')